이미 시간순으로 정렬되어있는 데이터라 정렬은 따로 손 안봐줌.

# Library

In [5]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import FinanceDataReader as fdr

import datetime as dt

import numpy as np

from tqdm import tqdm #진행바
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

# GPU
import tensorflow as tf

#https://www.tensorflow.org/guide/gpu#allowing_gpu_memory_growth
#프로세스의 요구량만큼 메모리 사용 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
        
#지정해서 쓰고 싶으면 model 쓸 때는 with 구문 안에 넣기

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# 방법1) not with open

## cd_dt_dic, x, y - 이게 13초 더 빠름.

In [2]:
%%time

# data load
data = pd.read_csv('stock_2018.csv')
data = data.drop('Index', axis=1)


# col names
col_arr = data.columns.values
col_arr
len(col_arr)

# 설명 idx 필요 없음, 식별변수 필요해서 0번째 열부터 써야함.
목적_idx = np.where(col_arr == 'Next_Change')[0][0] # 목적변수 열 idx

# 사실 굳이 나눌 필요는 없긴 함. 처음에 코드 쓸 때 나눠서 썼으니 계속 나눠서 쓰는것..
x = pd.concat([data.iloc[:, 0:목적_idx], data.iloc[:,목적_idx+1:]], axis = 1)
y = pd.concat([data.iloc[:, 0:2], data[['Next_Change']]], axis = 1)


# 여기부터 with open문 안으로 들어가면 될 것 같음.
# (x[['Trading_Value']] >= 1000_0000_0000).sum() #2682
천억 = x[x['Trading_Value'] >= 1000_0000_0000]
# need_date = list(np.sort(천억['Date'].unique()))


# 코드랑 날짜 같이 부르기
cd_dt_dic = {}
for i in range(len(천억[['Code','Date']])):
    
    tmp = 천억[['Code','Date']].iloc[[i]]
    tmp_cd = tmp['Code'].values[0]
    tmp_dt = tmp['Date'].values[0]
    
    # 사전에 key가 이미 있으면 해당 키의 val list에 값 append하기 # 뭔가 함수가 있을 줄 알았는데 없었다. update는 key에 매치된 val값을 새로 바꾸는거..
    if tmp_cd not in cd_dt_dic:
        globals()[f'data_{tmp_cd}'] = [tmp_dt]
        cd_dt_dic[tmp_cd] = globals()[f'data_{tmp_cd}']
    else:
        globals()[f'data_{tmp_cd}'].append(tmp_dt)
        cd_dt_dic[tmp_cd] = globals()[f'data_{tmp_cd}']
# cd_dt_dic
x

CPU times: user 2.37 s, sys: 135 ms, total: 2.51 s
Wall time: 2.51 s


,Code,Date,Open,High,Low,Close,Volume,Trading_Value,Change,MA5,...,SRSI,TSI,UO,SR,WR,AO,KAMA,ROC,PPO,PVO
0,50120,2018-01-02,10250,12050,10150,11800,26086769,307823874200,0.145631,10108.0,...,0.617452,22.3595,45.6450,73.6842,-26.3158,1259.470,9897.76,44.78530,6.24021,42.05600
1,50120,2018-01-03,11950,12450,10900,11750,20460474,240410569500,-0.004237,10538.0,...,0.606881,24.6104,44.5857,72.7096,-27.2904,1435.350,10067.00,44.17180,7.16213,36.42410
2,50120,2018-01-04,11850,14150,11600,12600,60663854,764364560400,0.072340,11232.0,...,0.691669,27.8890,46.0102,74.7145,-25.2855,1988.650,10221.40,53.09840,8.40391,42.40620
3,50120,2018-01-05,12800,13200,12000,12200,13935258,170010147600,-0.031746,11730.0,...,0.605418,28.9426,46.8657,67.9276,-32.0724,2473.210,10308.60,51.17720,8.92074,34.04080
4,50120,2018-01-08,12450,13400,12350,12850,16471707,211661434950,0.053279,12240.0,...,0.670468,30.8389,48.7672,78.6184,-21.3816,2793.380,10359.80,26.60100,9.65555,27.63640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380584,540,2018-12-21,4480,4480,4330,4480,109164,489054720,0.000000,4573.0,...,0.000000,-48.1839,40.9134,21.4286,-78.5714,-363.397,4592.44,-10.31030,-3.14961,2.96624
380585,540,2018-12-24,4480,4530,4440,4520,36973,167117960,0.008929,4534.0,...,0.250801,-48.6931,47.1329,27.1429,-72.8571,-394.015,4580.40,-8.87097,-3.21322,3.55599
380586,540,2018-12-26,4515,4515,4275,4470,28747,128499090,-0.011062,4512.0,...,0.177769,-49.7125,54.1570,27.2727,-72.7273,-422.632,4560.38,-7.83505,-3.31474,2.02404
380587,540,2018-12-27,4475,4500,4410,4470,20617,92157990,0.000000,4484.0,...,0.291459,-50.5188,54.6350,28.4672,-71.5328,-430.515,4538.92,-6.58307,-3.35971,-1.25023


In [6]:
%%time # 2분 46초 가량 더 느림.

dday_alldf = pd.DataFrame()

# need_date가 아닌 cd_dt_dic에서 고르기
for k,(key, val) in tqdm(enumerate(cd_dt_dic.items())):
#     key인 동안에
    for i in val:
        if np.where(x[x['Code']==key]['Date'].values == i)[0][0] >= 9 :
            idx = x[x['Code']==key][x['Date']==i].index

            dday_df = pd.DataFrame([x.iloc[:,0:2].loc[idx.values[0]]])
            dday_df.rename(index = {idx.values[0] : i}, inplace = True) # dday_df의 code와 같은 code인 df(x)중에서 고르기

            for j in range(9, -1, -1):

                tmp = x.iloc[idx-j].iloc[:,2:]
                tmp.columns = [f'D-{j}_{ori_col}' for ori_col in x.iloc[:,2:].columns]
                tmp = tmp.rename(index = {idx.values[0]-j : i})
        
                dday_df = pd.concat([dday_df, tmp], axis = 1)

            dday_df = pd.concat([dday_df, y.iloc[idx]['Next_Change'].rename(index = {idx.values[0]:i})], axis = 1)
            dday_alldf = pd.concat([dday_alldf, dday_df])

0it [00:00, ?it/s]

CPU times: user 7min 28s, sys: 4min 1s, total: 11min 29s
Wall time: 11min 30s


# 방법2) with open

## cd_dt_dic, x_df, y_df - 이게 13초 더 느림.

In [2]:
%%time

cd_dt_dic = {}
x_df = []
y_df = []

f = open('stock_2018.csv')
lines = f.readlines()
for i,line in tqdm(enumerate(lines)):
    line = line.strip()
    lst = np.array(line.split(','))
    lst = lst[1:]

    if i == 0 : # colname에서 원하는 거 (Next_Change idx) 뽑아서 변수에 저장. ; i==0일 때 lst == 위에서의 data.columns.values
        목적_idx = np.where(lst == 'Next_Change')[0][0] # 목적변수 열 idx

        x0 = np.append(lst[0:목적_idx], lst[목적_idx+1:])
        y0 = np.append(lst[0:2], lst[목적_idx])
        
        TV_idx = np.where(x0 == 'Trading_Value')[0][0] # 조건에 쓸 TV idx
    
    else:
        x1 = np.append(lst[0:목적_idx], lst[목적_idx+1:])
        y1 = np.append(lst[0:2], lst[목적_idx])
        
        x_df.append(x1)
        y_df.append(y1)
        
#         x1[TV_idx] #문자로 인식됨.
        if int(x1[TV_idx]) >= 1000_0000_0000:
        
            # 코드랑 날짜 같이 부르기
            tmp_cd = x1[0]
            tmp_dt = x1[1]
        
            # 사전에 key가 이미 있으면 해당 키의 val list에 값 append하기
            if tmp_cd not in cd_dt_dic: # cd_dt_dic = {} 위치가 아예 밖에 있어야지... 안에 있음 초기화되니까....
                globals()[f'data_{tmp_cd}'] = [tmp_dt]
                cd_dt_dic[tmp_cd] = globals()[f'data_{tmp_cd}']

            else:
                globals()[f'data_{tmp_cd}'].append(tmp_dt)
                cd_dt_dic[tmp_cd] = globals()[f'data_{tmp_cd}']
    
f.close()

0it [00:00, ?it/s]

CPU times: user 6.75 s, sys: 694 ms, total: 7.44 s
Wall time: 7.31 s


In [3]:
%%time

x_df = pd.DataFrame(x_df, columns = x0)
y_df = pd.DataFrame(y_df, columns = y0)

CPU times: user 8.49 s, sys: 565 ms, total: 9.05 s
Wall time: 9.05 s


In [5]:
%%time # 2분 46초 가량 더 빠름

dday_alldf = pd.DataFrame()

# need_date가 아닌 cd_dt_dic에서 고르기
for k,(key, val) in tqdm(enumerate(cd_dt_dic.items())):
#     key인 동안에
    for i in val:
        if np.where(x_df[x_df['Code']==key]['Date'].values == i)[0][0] >= 9 :
            idx = x_df[x_df['Code']==key][x_df['Date']==i].index
            
            #dday_df : 한 date에 대한 data 한 줄
            dday_df = pd.DataFrame([x_df.iloc[:,0:2].loc[idx.values[0]]])
            dday_df.rename(index = {idx.values[0] : i}, inplace = True) # dday_df의 code와 같은 code인 df(x)중에서 고르기

            for j in range(9, -1, -1):

                tmp = x_df.iloc[idx-j].iloc[:,2:]
                tmp.columns = [f'D-{j}_{ori_col}' for ori_col in x_df.iloc[:,2:].columns]
                tmp = tmp.rename(index = {idx.values[0]-j : i})
        
                dday_df = pd.concat([dday_df, tmp], axis = 1) # dday_df 옆으로 concat
            
            # dday_df 옆에 같은 idx의 y 붙여주기
            dday_df = pd.concat([dday_df, y_df.iloc[idx]['Next_Change'].rename(index = {idx.values[0]:i})], axis = 1)
            
            # dday_alldf: data 아래로 이어붙이기
            dday_alldf = pd.concat([dday_alldf, dday_df])            

0it [00:00, ?it/s]

<timed exec>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


CPU times: user 8min 44s, sys: 67.4 ms, total: 8min 44s
Wall time: 8min 44s


In [71]:
dday_alldf

,Code,Date,D-9_Open,D-9_High,D-9_Low,D-9_Close,D-9_Volume,D-9_Trading_Value,D-9_Change,D-9_MA5,...,D-0_TSI,D-0_UO,D-0_SR,D-0_WR,D-0_AO,D-0_KAMA,D-0_ROC,D-0_PPO,D-0_PVO,Next_Change
2018-01-15,050120,2018-01-15,10250,12050,10150,11800,26086769,307823874200,0.145631,10108,...,29.3225,45.3787,63.9228,-36.0772,3645.88,11858.8,68.6747,12.3246,38.852,-0.0681818
2018-01-16,050120,2018-01-16,11950,12450,10900,11750,20460474,240410569500,-0.00423729,10538,...,28.0689,45.6406,53.252,-46.748,3980.74,11947.6,47.7858,12.2902,30.6275,-0.188153
2018-01-17,050120,2018-01-17,11850,14150,11600,12600,60663854,764364560400,0.0723404,11232,...,22.3139,41.0354,25.813,-74.187,3687.65,11946.2,13.1068,10.4433,23.7,0.0472103
2018-01-18,050120,2018-01-18,12800,13200,12000,12200,13935258,170010147600,-0.031746,11730,...,18.8794,41.3833,28.8725,-71.1275,2987.65,11947.9,3.38983,9.22399,17.4172,0.0122951
2018-01-19,050120,2018-01-19,12450,13400,12350,12850,16471707,211661434950,0.0532787,12240,...,16.5383,33.5502,25,-75,2637.35,11949.9,5.10638,8.26069,12.6972,0.137652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-05-15,011760,2018-05-15,21400,24100,20800,22400,1488904,33351449600,0.0953545,20830,...,56.6722,68.0251,84.6369,-15.3631,6866.91,27722.5,74.9373,13.0112,68.4751,-0.0974212
2018-05-17,011760,2018-05-17,23500,23850,21950,22350,489326,10936436100,-0.0648536,22040,...,54.5403,64.0099,85.1942,-14.8058,10338.970588235294,29517.6,83.1296,15.4888,67.1178,0.0280374
2018-06-07,011760,2018-06-07,41650,41900,37000,38800,1305256,50643932800,-0.0582524,39190,...,29.5457,49.8274,70.6329,-29.3671,10088.1,36598.8,11.8182,13.029,17.827,-0.00116144
2018-06-11,011760,2018-06-11,40500,41300,40050,41300,712521,29427117300,0.298742,38620,...,31.8637,53.8917,95.8333,-4.16667,10979.7,39093.7,20.8738,13.9068,19.414,-0.0381526


# to_csv

In [6]:
dday_alldf.to_csv('stock_2018_10days.csv', index = False)
# pd.read_csv('stock_2018_10days.csv')

,Code,Date,D-9_Open,D-9_High,D-9_Low,D-9_Close,D-9_Volume,D-9_Trading_Value,D-9_Change,D-9_MA5,...,D-0_TSI,D-0_UO,D-0_SR,D-0_WR,D-0_AO,D-0_KAMA,D-0_ROC,D-0_PPO,D-0_PVO,Next_Change
0,50120,2018-01-15,10250,12050,10150,11800,26086769,307823874200,0.145631,10108.0,...,29.3225,45.3787,63.9228,-36.07720,3645.880000,11858.8,68.67470,12.32460,38.8520,-0.068182
1,50120,2018-01-16,11950,12450,10900,11750,20460474,240410569500,-0.004237,10538.0,...,28.0689,45.6406,53.2520,-46.74800,3980.740000,11947.6,47.78580,12.29020,30.6275,-0.188153
2,50120,2018-01-17,11850,14150,11600,12600,60663854,764364560400,0.072340,11232.0,...,22.3139,41.0354,25.8130,-74.18700,3687.650000,11946.2,13.10680,10.44330,23.7000,0.047210
3,50120,2018-01-18,12800,13200,12000,12200,13935258,170010147600,-0.031746,11730.0,...,18.8794,41.3833,28.8725,-71.12750,2987.650000,11947.9,3.38983,9.22399,17.4172,0.012295
4,50120,2018-01-19,12450,13400,12350,12850,16471707,211661434950,0.053279,12240.0,...,16.5383,33.5502,25.0000,-75.00000,2637.350000,11949.9,5.10638,8.26069,12.6972,0.137652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2532,11760,2018-05-15,21400,24100,20800,22400,1488904,33351449600,0.095354,20830.0,...,56.6722,68.0251,84.6369,-15.36310,6866.910000,27722.5,74.93730,13.01120,68.4751,-0.097421
2533,11760,2018-05-17,23500,23850,21950,22350,489326,10936436100,-0.064854,22040.0,...,54.5403,64.0099,85.1942,-14.80580,10338.970588,29517.6,83.12960,15.48880,67.1178,0.028037
2534,11760,2018-06-07,41650,41900,37000,38800,1305256,50643932800,-0.058252,39190.0,...,29.5457,49.8274,70.6329,-29.36710,10088.100000,36598.8,11.81820,13.02900,17.8270,-0.001161
2535,11760,2018-06-11,40500,41300,40050,41300,712521,29427117300,0.298742,38620.0,...,31.8637,53.8917,95.8333,-4.16667,10979.700000,39093.7,20.87380,13.90680,19.4140,-0.038153
